In [ ]:
from xgboost import XGBRegressor
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
import sklearn

import tensorflow as tf

from utils import *

warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [ ]:
# Importing the dataset
dataset = pd.read_csv('data.csv')

# Shuffle the dataset
dataset = dataset.sample(frac=1)

# Drop ZIP code column
dataset = dataset.drop(['ZIP'], axis=1)

# Convert #3 START_TIME (pandas Series Object) to datetime without date
dataset['START_TIME'] = pd.to_datetime(dataset['START_TIME']).apply(lambda x: x.value)

# drops index
try:
    dataset = dataset.drop(['INDEX'], axis=1)
except:
    pass

dataset.head()
dataset.info()

In [ ]:
x = dataset.iloc[:,0:-1].values
y = dataset.iloc[:, -1].values
x,y

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)
x

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.10, random_state=1)

In [ ]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = x.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256/2, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256/4, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=[r2metrics, soft_acc, 'mean_absolute_error', 'mean_squared_error'])
NN_model.summary()

In [ ]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [ ]:
NN_model.fit(x_train, y_train, epochs=350, batch_size=12, validation_split = .05, callbacks=[checkpoint])

In [ ]:
score = NN_model.evaluate(x_test, y_test)
print(score)